In [1]:
import os

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

from functools import partial

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

from configs import Inputs

import torchvision
from torchvision import datasets
from torchvision import transforms

import numpy as np

In [2]:
DEVICE = torch.device("cuda:0" if not torch.cuda.is_available() else "cpu")
batch_size = 4
CLASSES = 2
DIR = Inputs().DATASET_DIR
EPOCHS = 1
N_TRAIN_EXAMPLES = batch_size * 32
N_VALID_EXAMPLES = batch_size * 8
gpus_per_trial = 1
cpus_per_trial = 4

print(f"Device: {DEVICE}\nBatch size: {batch_size}\nClasses: {CLASSES}\n\
Dir: {DIR}\nEpochs: {EPOCHS}\n\
Number of training examples: {N_TRAIN_EXAMPLES}\n\
Number of validation examples: {N_VALID_EXAMPLES}\nGPUs/trial: {gpus_per_trial}")

Device: cpu
Batch size: 4
Classes: 2
Dir: /home/bernardo/github/sex-age-estimation/backup-bia/patch-1/data/pan-radiographs
Epochs: 1
Number of training examples: 128
Number of validation examples: 32
GPUs/trial: 1


In [3]:
from torchvision import transforms as T
img_size = 224

transform = T.Compose([
                T.Resize((img_size,img_size)),
                T.ToTensor(),
                T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
            ])

In [4]:
from torch.utils.data import DataLoader
from configs import Inputs
from utils.augmentations import get_transforms
from utils.data import FullRadiographSexDataset

val_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                       fold_nums=Inputs().val_folds,
                                       transforms=get_transforms(Inputs(), subset=["val"]))

val_dataloader = DataLoader(val_dataset,
                            batch_size=batch_size,
                            shuffle=False,
                            num_workers=0)

train_dataset = FullRadiographSexDataset(root_dir=Inputs.DATASET_DIR,
                                         fold_nums=Inputs().train_folds,
                                         transforms=get_transforms(Inputs(), subset=["train"]))

train_dataloader = DataLoader(train_dataset,
                              batch_size=batch_size,
                              shuffle=False,
                              num_workers=0)

Using only horizontal flip augmentation.
Using only horizontal flip augmentation.


In [5]:
config = {
    "lr": tune.loguniform(1e-5, 1e-1),
    "optimizer_name": tune.choice(["Adam", "RMSprop", "SGD"]),
    "n_neuronios_camada_final": tune.sample_from(lambda _: 2**np.random.randint(2, 10))
}

In [6]:
def criarModelo(n_neuronios):
    model = torchvision.models.efficientnet_b0(weights=torchvision.models.EfficientNet_B0_Weights.DEFAULT)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, n_neuronios)
    model.classifier.append(nn.Linear(n_neuronios, 2))
    return model

In [7]:
def test_accuracy(net):
    trainset, testset = train_dataset, val_dataset

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [8]:
def objective(config):

    # Gerar o modelo
    model = criarModelo(config['n_neuronios_camada_final'])
    
    if torch.cuda.is_available() and torch.cuda.device_count() > 1:
            model = nn.DataParallel(model)
    model.to(DEVICE)
    
    # Gerar optimizer
    optimizer_name = config['optimizer_name']
    lr = config["lr"]
    
    print("opt_name:", optimizer_name, "\nlr:", lr)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    train_loader, valid_loader = train_dataloader, val_dataloader

    criterion = F.nll_loss

    for epoch in range(10):
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(train_dataloader, 0):
            inputs, labels = data
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if epoch_steps % 5 == 0:
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
            running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        accuracy = test_accuracy(model)

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((model.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=loss, accuracy=accuracy)

    return accuracy

In [9]:
scheduler = ASHAScheduler(
                metric="loss",
                mode="min",
                max_t=EPOCHS,
                grace_period=1,
                reduction_factor=2)

reporter = CLIReporter(
                parameter_columns=["n_neuronios_camada_final", "lr", "optimizer_name"],
                metric_columns=["loss", "accuracy", "training_iteration"])

result = tune.run(partial(objective),
                  resources_per_trial={"cpu": cpus_per_trial, "gpu": gpus_per_trial},
                  config=config,
                  num_samples=N_TRAIN_EXAMPLES,
                  scheduler=scheduler,
                  progress_reporter=reporter,
)

best_trial = result.get_best_trial("loss", "min", "last")
print("Best trial config: {}".format(best_trial.config))
print("Best trial final validation loss: {}".format(
best_trial.last_result["loss"]))
print("Best trial final validation accuracy: {}".format(
best_trial.last_result["accuracy"]))

best_trained_model = criarModelo(config['n_neuronios_camada_final'])
if torch.cuda.is_available() and gpus_per_trial > 1:
        best_trained_model = nn.DataParallel(best_trained_model)
best_trained_model.to(DEVICE)

best_checkpoint_dir = best_trial.checkpoint.value
model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
best_trained_model.load_state_dict(model_state)

test_acc = test_accuracy(best_trained_model)
print("Best trial test set accuracy: {}".format(test_acc))

2022-12-22 22:40:51,042	INFO worker.py:1518 -- Started a local Ray instance.
2022-12-22 22:40:51,992	WARNING function_trainable.py:619 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


== Status ==
Current time: 2022-12-22 22:40:52 (running for 00:00:00.24)
Memory usage on this node: 2.7/15.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 1.000: None
Resources requested: 4.0/8 CPUs, 1.0/1 GPUs, 0.0/8.35 GiB heap, 0.0/4.17 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-22_22-40-52
Number of trials: 16/128 (15 PENDING, 1 RUNNING)
+-----------------------+----------+-----------------+------------------------+-------------+------------------+
| Trial name            | status   | loc             |   n_neuronios_camad... |          lr | optimizer_name   |
|-----------------------+----------+-----------------+------------------------+-------------+------------------|
| objective_b04b2_00000 | RUNNING  | 172.17.0.2:4703 |                     16 | 0.0176229   | RMSprop          |
| objective_b04b2_00001 | PENDING  |                 |                      4 | 4.49517e-05 | SGD              |
| objective_b04b2_00002 | PENDING 

(raylet) [2022-12-22 22:41:01,031 E 4410 4430] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-22_22-40-49_179257_4345 is over 95% full, available space: 15536992256; capacity: 455568355328. Object creation will fail if spilling is required.


== Status ==
Current time: 2022-12-22 22:41:04 (running for 00:00:12.73)
Memory usage on this node: 4.4/15.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 1.000: None
Resources requested: 4.0/8 CPUs, 1.0/1 GPUs, 0.0/8.35 GiB heap, 0.0/4.17 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-22_22-40-52
Number of trials: 17/128 (16 PENDING, 1 RUNNING)
+-----------------------+----------+-----------------+------------------------+-------------+------------------+
| Trial name            | status   | loc             |   n_neuronios_camad... |          lr | optimizer_name   |
|-----------------------+----------+-----------------+------------------------+-------------+------------------|
| objective_b04b2_00000 | RUNNING  | 172.17.0.2:4703 |                     16 | 0.0176229   | RMSprop          |
| objective_b04b2_00001 | PENDING  |                 |                      4 | 4.49517e-05 | SGD              |
| objective_b04b2_00002 | PENDING 

(raylet) [2022-12-22 22:41:11,037 E 4410 4430] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-22_22-40-49_179257_4345 is over 95% full, available space: 15536984064; capacity: 455568355328. Object creation will fail if spilling is required.


...

(raylet) [2022-12-22 22:51:21,400 E 4410 4430] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-22_22-40-49_179257_4345 is over 95% full, available space: 15536410624; capacity: 455568355328. Object creation will fail if spilling is required.


== Status ==
Current time: 2022-12-22 22:51:25 (running for 00:10:33.56)
Memory usage on this node: 4.9/15.6 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 1.000: None
Resources requested: 4.0/8 CPUs, 1.0/1 GPUs, 0.0/8.35 GiB heap, 0.0/4.17 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/objective_2022-12-22_22-40-52
Number of trials: 17/128 (16 PENDING, 1 RUNNING)
+-----------------------+----------+-----------------+------------------------+-------------+------------------+
| Trial name            | status   | loc             |   n_neuronios_camad... |          lr | optimizer_name   |
|-----------------------+----------+-----------------+------------------------+-------------+------------------|
| objective_b04b2_00000 | RUNNING  | 172.17.0.2:4703 |                     16 | 0.0176229   | RMSprop          |
| objective_b04b2_00001 | PENDING  |                 |                      4 | 4.49517e-05 | SGD              |
| objective_b04b2_00002 | PENDING 

(raylet) [2022-12-22 22:51:31,406 E 4410 4430] (raylet) file_system_monitor.cc:105: /tmp/ray/session_2022-12-22_22-40-49_179257_4345 is over 95% full, available space: 15503790080; capacity: 455568355328. Object creation will fail if spilling is required.


(func pid=4996) opt_name: SGD 
(func pid=4996) lr: 4.4951732008717886e-05


RuntimeError: Can't call numpy() on Tensor that requires grad. Use tensor.detach().numpy() instead.

In [ ]:
## TODO
# 1 - Adicionar modificações da última camada para um modelo pequeno no hyperbanding.
# 2 - Investigar como estamos retornando os melhores parâmetros.
# 3 - Ler o artigo do hyperbanding.

## Prioridade baixa
# Descobrir o que o reduction_factor faz.